# Examples 2

In [10]:
%matplotlib notebook
import numpy as np
import pandas as pd
import geopandas as gpd
import cartopy as cp
import xarray as xr
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
from utils import * # import some of my own functions
import fsspec
import s3fs

## Shapefiles

Shapefiles are files that contain vectors (shapes) geo-referenced to a particular coordinate system

They are often used in hydrology to define catchment areas, river channels and hydrological administrative regions. 

Each 'shape' in a shapefile will typically have a set of attributes that tell you more information about the shape, such as the name of the river or catchment it defines. 

The easiest way to see what's in a shapefile in python is to use a package called geopandas. It is essentially an extension of the pandas package, which is used to work with tabular data.

Sidenote: Shapefiles often confusingly come with several ancillary files. The main shapefile will have a **'.shp'** ending, with ancillaries ending with some or all of **'shx', '.sbx', '.sbn', '.dbf', '.cpg', '.prj'**, which provide additional information about the vectors/shapes contained in the **'.shp'** file. Most shapefile packages in python and elsewhere will read in the information they need from these ancillary files automatically if you provide the path to the **'.shp'** file, and therefore most of the time you can ignore them! If reading the shapefiles from an S3 object storage filesystem, like we are doing below, the ancillary files need to be zipped up together with the **'.shp'** file and the zip archive read in for the ancillaries to be automatically loaded. 

In [17]:
# Set up the S3 (object storage) filesystem object
s3 = s3fs.S3FileSystem(anon=True, endpoint_url="https://fdri-o.s3-ext.jc.rl.ac.uk")

In [22]:
# Read from the filesystem - note the zip as explained above
shapefile = gpd.read_file(s3.open('s3://example-data/gb_catchments.zip'))

In [23]:
shapefile

,OBJECTID,ID_STRING,ID,SHAPE_AREA,SHAPE_LEN,geometry
0,227,18010,18010.0,0.0,0.0,"POLYGON ((271449.998 695224.998, 271400 695224..."
1,244,21014,21014.0,0.0,0.0,"POLYGON ((310900.002 628474.999, 311150.002 62..."
2,245,21015,21015.0,0.0,0.0,"POLYGON ((356475.002 638850.002, 356349.999 63..."
3,247,21017,21017.0,0.0,0.0,"POLYGON ((323424.998 613149.998, 323375 613100..."
4,269,22004,22004.0,0.0,0.0,"POLYGON ((421150.002 612925, 421225.001 612850..."
...,...,...,...,...,...,...
1888,174,21003,21003.0,0.0,0.0,"POLYGON ((325775.002 640050, 325775.002 640000..."
1889,187,19021,19021.0,0.0,0.0,"POLYGON ((333800.001 667825.002, 333900.002 66..."
1890,197,20806,20806.0,0.0,0.0,"POLYGON ((363849.998 677374.998, 364024.998 67..."
1891,204,17004,17004.0,0.0,0.0,"POLYGON ((333000 699774.998, 333100.001 699774..."


The tabular representation of the shapefile shows each shape in the shapefile as a separate row. Each column shows an attribute of the shapes, with the actual vector geometry stored in the final column.

We can select out one of the rows and plot it to see what it looks like: